# The Battle of Neighborhoods (Week 1)

# In this section we import the libraries that will be required to process the data.


We will import the required libraries for python.
pandas and numpy for handling data. request module for using FourSquare API. geopy to get co-ordinates of City of New York. folium to visualize the results on a map

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Download and Explore Dataset

Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

We can find this dataset  on the web: https://geo.nyu.edu/catalog/nyu_2451_34572

I've download the json version file on my Jypter localhost

In [2]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
#!wget -q -O 'nyu-2451_34572-geojson.json' https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json
#print('Data downloaded!')

with open('nyu-2451-34572-geojson.json','r') as json_data:
    newyork_data = json.load(json_data)

Transfrom the data into pandas dataframe

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Create a map of New York with neighborhoods superimposed on top.

In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Foursquare venues

In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [11]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'JDCCUU2BZRLUK4QFZDGOLDZ2QLPG1YOPLCLR2Y20T4CA4L2S' # your Foursquare ID
CLIENT_SECRET = 'NF51FIR04YXJTPDMXIJUMX153JI0GVSULXOXZ0TILTHTTM3Q'# your Foursquare Secret
VERSION = '20181020'

In [12]:
#https://developer.foursquare.com/docs/resources/categories
#bakery = 4bf58dd8d48988d16a941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_bakery = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16a941735')
newyork_venues_bakery.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Auntie Anne's,40.877400,-73.904947,Bakery
1,Marble Hill,40.876551,-73.91066,Franco #2 Bakery,40.874975,-73.909381,Food
2,Marble Hill,40.876551,-73.91066,Rosarina Bakery (aka Franco Bakery),40.874870,-73.909398,Bakery
3,Marble Hill,40.876551,-73.91066,Cafeccino Bakery,40.880209,-73.906884,Bagel Shop
4,Marble Hill,40.876551,-73.91066,Corner Cafe And Bakery,40.887248,-73.906822,Bakery


In [13]:
newyork_venues_bakery.shape

(1575, 7)

In [14]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [15]:
map_newyork_bakery = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_bakery, 'red', map_newyork_bakery)
map_newyork_bakery

In [17]:
newyork_venues_bakery.shape

(1575, 7)

In [18]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
manhattan_grouped = newyork_venues_bakery.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,39,39,39,39,39,39
Carnegie Hill,47,47,47,47,47,47
Central Harlem,14,14,14,14,14,14
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,49,49,49,49,49,49
East Harlem,42,42,42,42,42,42
East Village,46,46,46,46,46,46


# 3. Analyze Each Neighborhood

In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_bakery[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_bakery['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Arcade,Bagel Shop,Bakery,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cupcake Shop,Deli / Bodega,Dessert Shop,Donut Shop,Farmers Market,Food,French Restaurant,Gourmet Shop,Ice Cream Shop,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Pie Shop,Pizza Place,Sandwich Place,Snack Place,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,American Restaurant,Arcade,Bagel Shop,Bakery,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cupcake Shop,Deli / Bodega,Dessert Shop,Donut Shop,Farmers Market,Food,French Restaurant,Gourmet Shop,Ice Cream Shop,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Pie Shop,Pizza Place,Sandwich Place,Snack Place,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.025641,0.000000,0.000000,0.717949,0.00,0.000000,0.051282,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.051282,0.000000,0.000000,0.0,0.000000,0.025641,0.051282,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025641,0.025641,0.000000,0.000000
1,Carnegie Hill,0.042553,0.000000,0.000000,0.744681,0.00,0.000000,0.085106,0.000000,0.000000,0.000000,0.021277,0.021277,0.000000,0.021277,0.000000,0.000000,0.0,0.000000,0.021277,0.021277,0.000000,0.021277,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.857143,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.020000,0.720000,0.00,0.000000,0.080000,0.020000,0.000000,0.000000,0.060000,0.020000,0.000000,0.020000,0.000000,0.000000,0.0,0.000000,0.000000,0.040000,0.000000,0.020000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Chinatown,0.000000,0.000000,0.040000,0.740000,0.02,0.000000,0.020000,0.000000,0.020000,0.020000,0.020000,0.020000,0.000000,0.040000,0.020000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000
5,Civic Center,0.020000,0.000000,0.000000,0.680000,0.02,0.000000,0.080000,0.000000,0.020000,0.000000,0.040000,0.020000,0.000000,0.060000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000
6,Clinton,0.020408,0.000000,0.000000,0.775510,0.00,0.000000,0.040816,0.000000,0.000000,0.000000,0.040816,0.020408,0.000000,0.020408,0.020408,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.020408,0.020408,0.000000,0.020408,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.833333,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.047619,0.000000,0.023810,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.021739,0.000000,0.021739,0.760870,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.065217,0.043478,0.000000,0.065217,0.000000,0.000000,0.0,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Financial District,0.025641,0.025641,0.000000,0.666667,0.00,0.000000,0.076923,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.051282,0.000000,0.000000,0.0,0.000000,0.025641,0.051282,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025641,0.025641,0.000000,0.000000


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Bakery,Dessert Shop,Ice Cream Shop,Café,Coffee Shop,Gourmet Shop,American Restaurant,Sandwich Place,Snack Place,Breakfast Spot
1,Carnegie Hill,Bakery,Café,American Restaurant,Cupcake Shop,Coffee Shop,Gourmet Shop,Ice Cream Shop,Italian Restaurant,Dessert Shop,Deli / Bodega
2,Central Harlem,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
3,Chelsea,Bakery,Café,Coffee Shop,Ice Cream Shop,Dessert Shop,Bagel Shop,Italian Restaurant,Caribbean Restaurant,Cupcake Shop,Vegetarian / Vegan Restaurant
4,Chinatown,Bakery,Dessert Shop,Bagel Shop,Donut Shop,Sandwich Place,Pie Shop,Breakfast Spot,Café,Chinese Restaurant,Cocktail Bar


Cluster Neighborhoods

In [24]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 2, 2, 1, 2, 0, 2, 1], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Bakery,Bagel Shop,Food,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Bakery,Dessert Shop,Bagel Shop,Donut Shop,Sandwich Place,Pie Shop,Breakfast Spot,Café,Chinese Restaurant,Cocktail Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Bakery,Ice Cream Shop,Coffee Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Bakery,Bagel Shop,Food,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
2,Washington Heights,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
7,East Harlem,Bakery,Café,Ice Cream Shop,Mexican Restaurant,Italian Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Deli / Bodega,Arcade,Bagel Shop
11,Roosevelt Island,Bakery,Ice Cream Shop,French Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop
20,Lower East Side,Bakery,Bagel Shop,Donut Shop,Pie Shop,Cocktail Bar,Coffee Shop,Dessert Shop,Arcade,Breakfast Spot,Bubble Tea Shop
37,Stuyvesant Town,Bakery,Cupcake Shop,Dessert Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café
39,Hudson Yards,Bakery,Coffee Shop,Donut Shop,Cupcake Shop,Chinese Restaurant,Dessert Shop,Pie Shop,Pizza Place,Snack Place,Bagel Shop


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Bakery,Dessert Shop,Coffee Shop,Ice Cream Shop,Israeli Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Breakfast Spot,Bubble Tea Shop,Deli / Bodega
15,Midtown,Bakery,Coffee Shop,Café,Donut Shop,Japanese Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Pie Shop,Snack Place
18,Greenwich Village,Bakery,Coffee Shop,Café,Dessert Shop,Breakfast Spot,Cupcake Shop,French Restaurant,American Restaurant,Pie Shop,Bagel Shop
21,Tribeca,Bakery,Dessert Shop,Café,Breakfast Spot,Cupcake Shop,Coffee Shop,Ice Cream Shop,American Restaurant,Bagel Shop,Sandwich Place
22,Little Italy,Bakery,Dessert Shop,Café,Coffee Shop,Donut Shop,French Restaurant,Bagel Shop,Breakfast Spot,Chinese Restaurant,Cupcake Shop
23,Soho,Bakery,Dessert Shop,Café,Coffee Shop,Cupcake Shop,French Restaurant,Chinese Restaurant,American Restaurant,Breakfast Spot,Pie Shop
27,Gramercy,Bakery,Dessert Shop,Coffee Shop,Café,Cupcake Shop,American Restaurant,Japanese Restaurant,Italian Restaurant,Arcade,Bagel Shop
29,Financial District,Bakery,Café,Dessert Shop,Ice Cream Shop,Coffee Shop,Gourmet Shop,American Restaurant,Arcade,Sandwich Place,Snack Place
31,Noho,Bakery,Coffee Shop,Bagel Shop,Donut Shop,Café,Cupcake Shop,Cocktail Bar,French Restaurant,Bubble Tea Shop,Pie Shop
32,Civic Center,Bakery,Café,Dessert Shop,Coffee Shop,Cupcake Shop,Chinese Restaurant,Ice Cream Shop,American Restaurant,Breakfast Spot,Sandwich Place


In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Dessert Shop,Bagel Shop,Donut Shop,Sandwich Place,Pie Shop,Breakfast Spot,Café,Chinese Restaurant,Cocktail Bar
8,Upper East Side,Bakery,Café,American Restaurant,Dessert Shop,Sandwich Place,Bagel Shop,Gourmet Shop,French Restaurant,Cupcake Shop,Arcade
9,Yorkville,Bakery,Café,Ice Cream Shop,Coffee Shop,French Restaurant,Gourmet Shop,American Restaurant,Sandwich Place,Dessert Shop,Bubble Tea Shop
10,Lenox Hill,Bakery,Dessert Shop,Cupcake Shop,Coffee Shop,French Restaurant,Gourmet Shop,Ice Cream Shop,American Restaurant,Bagel Shop,Sandwich Place
13,Lincoln Square,Bakery,Café,Dessert Shop,Cupcake Shop,Donut Shop,French Restaurant,Coffee Shop,American Restaurant,Italian Restaurant,Israeli Restaurant
14,Clinton,Bakery,Café,Coffee Shop,Donut Shop,Dessert Shop,Cupcake Shop,American Restaurant,Pie Shop,Pizza Place,Snack Place
16,Murray Hill,Bakery,Café,Coffee Shop,Japanese Restaurant,Cupcake Shop,Donut Shop,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot
17,Chelsea,Bakery,Café,Coffee Shop,Ice Cream Shop,Dessert Shop,Bagel Shop,Italian Restaurant,Caribbean Restaurant,Cupcake Shop,Vegetarian / Vegan Restaurant
19,East Village,Bakery,Dessert Shop,Coffee Shop,Cupcake Shop,American Restaurant,Bagel Shop,French Restaurant,Arcade,Breakfast Spot,Bubble Tea Shop
24,West Village,Bakery,Coffee Shop,Dessert Shop,Bagel Shop,Bubble Tea Shop,Café,Caribbean Restaurant,Cupcake Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant
4,Hamilton Heights,Bakery,Ice Cream Shop,Coffee Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café
6,Central Harlem,Bakery,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop,Café,Caribbean Restaurant


In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Bakery,Ice Cream Shop,French Restaurant,Coffee Shop,Farmers Market,Vegetarian / Vegan Restaurant,Deli / Bodega,Arcade,Bagel Shop,Breakfast Spot
25,Manhattan Valley,Bakery,Ice Cream Shop,Dessert Shop,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Arcade,Bagel Shop,Breakfast Spot,Bubble Tea Shop
26,Morningside Heights,Bakery,Ice Cream Shop,Dessert Shop,Farmers Market,Coffee Shop,French Restaurant,Chinese Restaurant,Cupcake Shop,Cocktail Bar,Vegetarian / Vegan Restaurant
